In [1]:
%%capture --no-stderr
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
import warnings
import nest_asyncio
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")
nest_asyncio.apply()
_ = load_dotenv(find_dotenv())

In [3]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")

## 1. Human in the loop two ways

### a. The `HumanResponseEvent` approach

In [4]:
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    Context,
    StartEvent,
    StopEvent
)
from llama_index.core.workflow.events import InputRequiredEvent, HumanResponseEvent
from llama_index.core.chat_engine import SimpleChatEngine

This workflow is as simple as it gets! We ask for the username because we only allow a specific user to invoke the workflow. So if the user is not `Titus`, we just simply won't run it. 

In [5]:
class AnswerEvent(Event):
    query: str

class AskLLM(Workflow):
    def __init__(self, llm = llm,  *args, **kwargs):
        self.user = None
        self.approved_users = ["Titus"]
        self.engine = SimpleChatEngine.from_defaults(llm=llm)
        super().__init__(*args, **kwargs)
    
    @step(pass_context=True)
    async def start(self, ctx: Context, ev: StartEvent) -> InputRequiredEvent | AnswerEvent:
        if self.user is None:
            print("Unidentified user. Verifying credentials...\n\n")
            await ctx.set("query", ev.query)
            return InputRequiredEvent(prefix="Please identify yourself: ")
        return AnswerEvent(query=ev.query)

    @step(pass_context=True)
    async def authenticate(self, ctx: Context, ev: HumanResponseEvent) -> AnswerEvent | StopEvent:
        user = ev.response
        if user in self.approved_users:
            self.user = user
            query = await ctx.get("query")
            return AnswerEvent(query=str(query))
        return StopEvent(result = "Unauthorized user. Please acquire authorization credentials from administrators.")
    
    @step
    async def answer(self, ev: AnswerEvent) -> StopEvent:
        response = self.engine.chat(ev.query)
        return StopEvent(result=str(response))

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(AskLLM, filename="AskLLM.html")

In [6]:
app1 = AskLLM()

handler1 = app1.run(query="What is OpenAI?")

## Use a for-loop to handle the `HumanResponseEvent`
async for event in handler1.stream_events():
    if isinstance(event, InputRequiredEvent):
        response = input(event.prefix)
        handler1.ctx.send_event(HumanResponseEvent(response=response))

final_result = await handler1

Unidentified user. Verifying credentials...




In [7]:
display(Markdown(final_result))

OpenAI is an artificial intelligence research organization that aims to develop and promote friendly AI for the benefit of humanity. Founded in December 2015, OpenAI conducts research in various areas of AI, including machine learning, natural language processing, robotics, and more. The organization is known for creating advanced AI models, such as the GPT (Generative Pre-trained Transformer) series, which includes models like ChatGPT.

OpenAI's mission is to ensure that artificial general intelligence (AGI) is aligned with human values and can be used safely and responsibly. The organization emphasizes transparency, collaboration, and ethical considerations in AI development. OpenAI also engages with the broader community through partnerships, publications, and open-source projects to foster a responsible approach to AI technology.

### b. The `StopEvent` approach

In [8]:
class AuthEvent(Event):
    user: str

class AskLLM_2(Workflow):
    def __init__(self, llm = llm,  *args, **kwargs):
        self.user = None
        self.approved_users = ["Titus"]
        self.engine = SimpleChatEngine.from_defaults(llm=llm)
        super().__init__(*args, **kwargs)
    
    @step(pass_context=True)
    async def start(self, ctx: Context, ev: StartEvent) -> StopEvent | AuthEvent:
        if ev.user is None:
            return StopEvent("Please furnish your user credentials.")
        await ctx.set("query", ev.query)
        return AuthEvent(user=ev.user)

    @step(pass_context=True)
    async def authenticate(self, ctx: Context, ev: AuthEvent) -> AnswerEvent | StopEvent:
        user = ev.user
        if user in self.approved_users:
            self.user = user
            query = await ctx.get("query")
            return AnswerEvent(query=str(query))
        return StopEvent(result = "Unauthorized user. Please acquire authorization credentials from administrators.")
    
    @step
    async def answer(self, ev: AnswerEvent) -> StopEvent:
        response = self.engine.chat(ev.query)
        return StopEvent(result=str(response))

In [16]:
draw_all_possible_flows(AskLLM_2, filename="AskLLM_2.html")

AskLLM_all.html


In [9]:
app2 = AskLLM_2()

response = await app2.run(query="What is a Large Language Model?", user=None)
display(Markdown(response))

Please furnish your user credentials.

In [10]:
response = await app2.run(query="What is a Large Language Model?", user="Bob")
display(Markdown(response))

Unauthorized user. Please acquire authorization credentials from administrators.

In [11]:
response = await app2.run(query="What is a Large Language Model?", user="Titus")

In [12]:
display(Markdown(response))

A Large Language Model (LLM) is a type of artificial intelligence model designed to understand, generate, and manipulate human language. These models are built using deep learning techniques, particularly neural networks, and are trained on vast amounts of text data from diverse sources, such as books, articles, websites, and more.

Key characteristics of LLMs include:

1. **Scale**: LLMs are characterized by their large number of parameters, often in the billions or even trillions. This scale allows them to capture complex patterns and nuances in language.

2. **Training**: They are trained using unsupervised or semi-supervised learning methods, where the model learns to predict the next word in a sentence given the previous words. This process helps the model understand grammar, context, and even some level of reasoning.

3. **Versatility**: LLMs can perform a wide range of language-related tasks, including text generation, translation, summarization, question answering, and more, often with little to no task-specific training.

4. **Contextual Understanding**: They can generate coherent and contextually relevant text based on the input they receive, making them useful for applications like chatbots, content creation, and interactive storytelling.

5. **Limitations**: Despite their capabilities, LLMs can produce incorrect or nonsensical answers, may reflect biases present in the training data, and lack true understanding or consciousness.

Examples of LLMs include OpenAI's GPT-3 and GPT-4, Google's BERT and T5, and Meta's LLaMA. These models have significantly advanced the field of natural language processing (NLP) and continue to evolve with ongoing research and development.

## 2. Deploying Workflows
LlamaIndex has many interesting Workflow implementations of research papers! Here's one of my favorites: the `Self-Discovery Workflow`, a "general framework for LLMs to self-discover the task-intrinsic reasoning structures to tackle complex reasoning problems that are challenging for typical prompting methods".

There are two stages for any given task:

STAGE-1:

a. SELECT: Selects subset of reasoning Modules.

b. ADAPT: Adapts selected reasoning modules to the task.

c. IMPLEMENT: It gives reasoning structure for the task.

STAGE-2: Uses the generated reasoning structure for the task to generate an answer.

> Reference: [Paper](https://arxiv.org/abs/2402.03620)

In [13]:
from llama_index.core.prompts import PromptTemplate

_REASONING_MODULES = [
    "1. How could I devise an experiment to help solve that problem?",
    "2. Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
    "3. How could I measure progress on this problem?",
    "4. How can I simplify the problem so that it is easier to solve?",
    "5. What are the key assumptions underlying this problem?",
    "6. What are the potential risks and drawbacks of each solution?",
    "7. What are the alternative perspectives or viewpoints on this problem?",
    "8. What are the long-term implications of this problem and its solutions?",
    "9. How can I break down this problem into smaller, more manageable parts?",
    "10. Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
    "11. Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
    "12. Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
    "13. Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
    "14. Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
    "15. Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
    "16. What is the core issue or problem that needs to be addressed?",
    "17. What are the underlying causes or factors contributing to the problem?",
    "18. Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
    "19. What are the potential obstacles or challenges that might arise in solving this problem?",
    "20. Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
    "21. Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
    "22. What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
    "23. How can progress or success in solving the problem be measured or evaluated?",
    "24. What indicators or metrics can be used?",
    "25. Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
    "26. Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
    "27. Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
    "28. Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
    "29. Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
    "30. Is the problem a design challenge that requires creative solutions and innovation?",
    "31. Does the problem require addressing systemic or structural issues rather than just individual instances?",
    "32. Is the problem time-sensitive or urgent, requiring immediate attention and action?",
    "33. What kinds of solution typically are produced for this kind of problem specification?",
    "34. Given the problem specification and the current best solution, have a guess about other possible solutions."
    "35. Let’s imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?"
    "36. What is the best way to modify this current best solution, given what you know about these kinds of problem specification?"
    "37. Ignoring the current best solution, create an entirely new solution to the problem."
    "38. Let’s think step by step ."
    "39. Let’s make a step by step plan and implement it with good notation and explanation.",
]

_REASONING_MODULES = "\n".join(_REASONING_MODULES)

SELECT_PRMOPT_TEMPLATE = PromptTemplate(
    "Given the task: {task}, which of the following reasoning modules are relevant? Do not elaborate on why.\n\n {reasoning_modules}"
)

ADAPT_PROMPT_TEMPLATE = PromptTemplate(
    "Without working out the full solution, adapt the following reasoning modules to be specific to our task:\n{selected_modules}\n\nOur task:\n{task}"
)

IMPLEMENT_PROMPT_TEMPLATE = PromptTemplate(
    "Without working out the full solution, create an actionable reasoning structure for the task using these adapted reasoning modules:\n{adapted_modules}\n\nTask Description:\n{task}"
)

REASONING_PROMPT_TEMPLATE = PromptTemplate(
    "Using the following reasoning structure: {reasoning_structure}\n\nSolve this task, providing your final answer: {task}"
)

### The Workflow

In [33]:
from llama_index.core.llms import LLM

class JudgeEvent(Event):
    task: str
    llm: LLM

class SetupEvent(Event):
    task: str

class GetModulesEvent(Event):
    """Event to get modules."""

    task: str
    modules: str


class RefineModulesEvent(Event):
    """Event to refine modules."""

    task: str
    refined_modules: str


class ReasoningStructureEvent(Event):
    """Event to create reasoning structure."""

    task: str
    reasoning_structure: str

In [ ]:
class SelfDiscoverWorkflow(Workflow):
    """Self discover workflow."""
    
    def __init__(self, llm: LLM):
        self.agent = SimpleChatEngine.from_defaults(llm=llm)
    
    
    @step
    async def judge_query(
        self, ctx: Context, ev: StartEvent
    ) -> StopEvent | SetupEvent:
        """Event to judge whether a query is well-written and to seek
        clarification if the query is not well-written."""
        
        current_chat_history = await ctx.get("chat_history", [])
        current_chat_history.append({"role": "user", "content": ev.task})
        
        response = await self.agent.chat(
            f"""
            Given a user query, determine if this is likely to yield good results from a RAG system as-is. If it's good, return 'good', if it's bad, return 'bad', if 
            it's not a question (for e.g. "Hi!", "Thanks!"), return "not a question".
            Good queries use a lot of relevant keywords and are detailed. Bad queries are vague or ambiguous.

            Here is the query: {ev.query}
            """
        )
        if response == "bad":
            final_response = await self.agent.chat(
                f"""
                The user has given a poor query. Seek for clarification from the user.
                
                Here's the user's query: {ev.query}
                """
            )   
            current_chat_history.append({"role": "assistant", "content": str(final_response)})
            await ctx.set("chat_history", current_chat_history)
            return StopEvent(str(final_response)) 
        
        elif response == "not a question":
            final_response = await self.agent.chat(
                f"""
                Here's the user's remark or greeting: {ev.query}
                
                Respond to the user politely.
                """
            )   
            current_chat_history.append({"role": "assistant", "content": str(final_response)})
            await ctx.set("chat_history", current_chat_history)
            return StopEvent(str(final_response)) 
        
        await ctx.set("chat_history", current_chat_history)
        return SetupEvent(task=ev.task)

    @step
    async def get_modules(
        self, ctx: Context, ev: SetupEvent
    ) -> GetModulesEvent:
        """Get modules step."""
        # get input data, store llm into ctx
        task = ev.get("task")

        if task is None or llm is None:
            raise ValueError("'task' and 'llm' arguments are required.")

        # format prompt and get result from LLM
        prompt = SELECT_PRMOPT_TEMPLATE.format(
            task=task, reasoning_modules=_REASONING_MODULES
        )
        result = self.llm.complete(prompt)

        return GetModulesEvent(task=task, modules=str(result))

    @step
    async def refine_modules(
        self, ctx: Context, ev: GetModulesEvent
    ) -> RefineModulesEvent:
        """Refine modules step."""
        task = ev.task
        modules = ev.modules
        llm: LLM = await ctx.get("llm")

        # format prompt and get result
        prompt = ADAPT_PROMPT_TEMPLATE.format(
            task=task, selected_modules=modules
        )
        result = llm.complete(prompt)

        return RefineModulesEvent(task=task, refined_modules=str(result))

    @step
    async def create_reasoning_structure(
        self, ctx: Context, ev: RefineModulesEvent
    ) -> ReasoningStructureEvent:
        """Create reasoning structures step."""
        task = ev.task
        refined_modules = ev.refined_modules
        llm: LLM = await ctx.get("llm")

        # format prompt, get result
        prompt = IMPLEMENT_PROMPT_TEMPLATE.format(
            task=task, adapted_modules=refined_modules
        )
        result = llm.complete(prompt)

        return ReasoningStructureEvent(
            task=task, reasoning_structure=str(result)
        )

    @step
    async def get_final_result(
        self, ctx: Context, ev: ReasoningStructureEvent
    ) -> StopEvent:
        """Gets final result from reasoning structure event."""
        task = ev.task
        reasoning_structure = ev.reasoning_structure
        llm: LLM = await ctx.get("llm")

        # format prompt, get res
        prompt = REASONING_PROMPT_TEMPLATE.format(
            task=task, reasoning_structure=reasoning_structure
        )
        result = llm.complete(prompt)

        return StopEvent(result=result)